In [1]:
import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange

from lib.data import conditions as cond
from lib.data import tables as tab
from lib import bootstrap as bs
from lib.features import correlation as corr
from lib.features import decomposition as de

In [2]:
engine = tab.sa.create_engine(os.environ['SQLALCHEMY_ENGINE_URL'])
Session = tab.sa.orm.sessionmaker(bind=engine)
session = Session()

In [3]:
def get_samplers(session, celltype) -> ("mouseid_sampler", "averagetr_sampler"):
    sample_mouseids = bs.get_mouseid_sampler(session, celltype)
    sample_cellids = {
        mid: (lambda mid=mid: bs.get_day_one_cell_ids(session, mid))
        for mid in bs.get_mouseids_by_celltype(session, celltype)
    }
    sample_averagetr = {
        day: {
            mid: bs.get_averagetrace_given_cellid_sampler(session, sample_cellids[mid], mid, day)
            for mid in bs.get_mouseids_by_celltype(session, celltype)
        }
        for day in (1, 7)
    }
    
    return sample_mouseids, sample_averagetr

In [4]:
trial_structure = de.TrialBasisFunctions(
    390, tone_duration=1.0, delay_duration=1.5, reward_duration=2.5
)
trial_bases = {
    'tone + delay': trial_structure.tone + trial_structure.delay,
    'reward': trial_structure.reward
}

Run bootstrap. Average shuffle traces are not Z-scored, but this doesn't matter because spearman correlation is scale invariant.

In [5]:
NUM_BOOTSTRAPS = 1000

# Pyramidal neurons

In [6]:
G

In [7]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

In [8]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,-0.004328
1,0,7,tone + delay,-0.022610
2,0,delta,tone + delay,-0.018282
3,0,1,reward,-0.014153
4,0,7,reward,0.017556
...,...,...,...,...
5995,999,7,tone + delay,0.003979
5996,999,delta,tone + delay,0.027601
5997,999,1,reward,-0.019756
5998,999,7,reward,-0.019996


In [9]:
if not os.path.exists('../../interneuron-reward-data/processed/pfc_corr_bootstrap.csv'):
    bs_df.to_csv('../../interneuron-reward-data/processed/pfc_corr_bootstrap.csv')
else:
    if input('Overwrite? (yn)').lower() == 'y':
        bs_df.to_csv('../../interneuron-reward-data/processed/pfc_corr_bootstrap.csv')

# VIP neurons

In [110]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.vip)

NaNs are introduced because one cell is missing from day 7 in VIP mouse CL146. It shows up as a row of NaNs in `sample_averagetr[7]['CL146']()`. Using `nanmean` here prevents this from crashing the bootstrap and doesn't affect the null distribution (except that the null distributions for day 7 and change are based on one less mouse, of course). Don't actually know which cell this is because cell ids are returned from `bs.get_day_one_cell_ids()` in no particular order.

In [111]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

<ipython-input-111-5eb2fae2e9c3>:15: RuntimeWarning: Mean of empty slice
  sp_stats_d7.append(np.nanmean(d7_tmp))
<ipython-input-111-5eb2fae2e9c3>:16: RuntimeWarning: Mean of empty slice
  sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.


In [112]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,0.033831
1,0,7,tone + delay,-0.025159
2,0,delta,tone + delay,-0.059395
3,0,1,reward,-0.017385
4,0,7,reward,-0.033826
...,...,...,...,...
5995,999,7,tone + delay,-0.032450
5996,999,delta,tone + delay,-0.002645
5997,999,1,reward,0.018676
5998,999,7,reward,-0.002388


In [113]:
if not os.path.exists('../../interneuron-reward-data/processed/vip_corr_bootstrap.csv'):
    bs_df.to_csv('../../interneuron-reward-data/processed/vip_corr_bootstrap.csv')
else:
    if input('Overwrite? (yn)').lower() == 'y':
        bs_df.to_csv('../../interneuron-reward-data/processed/vip_corr_bootstrap.csv')

# PV

In [114]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.pv)

In [115]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

In [116]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,-0.026677
1,0,7,tone + delay,0.035777
2,0,delta,tone + delay,0.062454
3,0,1,reward,0.027624
4,0,7,reward,0.013400
...,...,...,...,...
5995,999,7,tone + delay,0.023181
5996,999,delta,tone + delay,0.033517
5997,999,1,reward,-0.034347
5998,999,7,reward,-0.031531


In [117]:
if not os.path.exists('../../interneuron-reward-data/processed/pv_corr_bootstrap.csv'):
    bs_df.to_csv('../../interneuron-reward-data/processed/pv_corr_bootstrap.csv')
else:
    if input('Overwrite? (yn)').lower() == 'y':
        bs_df.to_csv('../../interneuron-reward-data/processed/pv_corr_bootstrap.csv')

---

# Scratch

In [140]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.pyr)

In [120]:
sample_averagetr

{1: {'CL136': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL144': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL146': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL188': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL199': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>},
 7: {'CL136': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL144': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL146': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  'CL188': <function lib.bootstrap.get_averagetrace_given_cellid_sampler.<locals>.sample() -> numpy.ndarray>,
  '

In [139]:
np.isnan(sample_averagetr[7]['CL146']()).sum(axis=1)

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0, 390,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [141]:
session.query(tab.Cell).filter(tab.Cell.mouse_id=='CL146').limit(10).all()

[<Cell 3423>,
 <Cell 3424>,
 <Cell 3425>,
 <Cell 3426>,
 <Cell 3427>,
 <Cell 3428>,
 <Cell 3429>,
 <Cell 3430>,
 <Cell 3431>,
 <Cell 3432>]